In [1]:
from scipy import signal
from sklearn import preprocessing
from scipy.signal import cheby2, freqz, filtfilt
from scipy.signal import decimate

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import os

In [2]:
#Functions

def Nan_interpolation(input_sig):
    nan_indices = np.isnan(input_sig)
    non_nan_indices = np.arange(len(input_sig))[~nan_indices]
    
    input_sig[nan_indices] = np.interp(np.where(nan_indices)[0], non_nan_indices, input_sig[non_nan_indices])
    
    return input_sig

#FIR BPF
def BPFilter(input_sig, lowcut=0.5, highcut=5.0, numtaps = 1001, window = 'hamming'):
    # Define filter parameters
    fs = 125 # Sampling frequency (Hz)
    nyquist = 0.5 * fs # Nyquist frequency
    bands = [lowcut/nyquist, highcut/nyquist]

    # Design the ppg FIR filter
    fir_coeff = signal.firwin(numtaps, bands, window = window, pass_zero = 'bandpass')

    # Apply the filter to the PPG signal
    filtered_sig = signal.lfilter(fir_coeff, 1.0, input_sig)[int((numtaps-1)/2):] #Filter delay
    
    return filtered_sig

def MAFilter(input_sig, window_size):
    window = np.ones(window_size) / window_size
    smoothed_signal = np.convolve(input_sig, window, mode='same')
    return smoothed_signal

def SAI_Algorithm(input_sig):
    # 1. Beat detection
    peaks_indx, _ = signal.find_peaks(input_sig, distance = 60)
    valleys_indx = np.array([], dtype=np.int64)
    valleys_indx = np.append(valleys_indx, [np.argmin(input_sig[peaks_indx[i]: peaks_indx[i + 1]]) + peaks_indx[i] for i in range(len(peaks_indx) - 1)])
    
    # 3. Abnormality criterion
    Ps_c = 180
    Pd_c = 20
    Pm_min_c, Pm_max_c = 30, 200
    f_min_c, f_max_c = 20, 200
    Pp_c = 20
    T = 2 #second
    dPs_c = 20
    dPd_c = 20
    dT_c = 2/3
    
    bp_ft_Ps = input_sig[peaks_indx[2:-1]]
    bp_ft_Pd = input_sig[valleys_indx[2:]]
    bp_ft_Pp = bp_ft_Ps - bp_ft_Pd
    bp_ft_Pm = (bp_ft_Ps + bp_ft_Pd) / 3
    bp_ft_T = (valleys_indx[2:] - valleys_indx[1:-1]) / 125
    bp_ft_f = 60 / bp_ft_T
    
    dPs = np.abs(bp_ft_Ps - input_sig[peaks_indx[1:-2]])
    dPd = np.abs(bp_ft_Pd - input_sig[valleys_indx[1:-1]])
    dT = np.abs(bp_ft_T - (valleys_indx[1:-1] - valleys_indx[:-2]) / 125)
    
    mask = (
        (bp_ft_Ps > Ps_c) |
        (bp_ft_Ps < 80) |
        (bp_ft_Pd < Pd_c) |
        (bp_ft_Pm < Pm_min_c) | (bp_ft_Pm > Pm_max_c) |
        (bp_ft_Pp < Pp_c) |
        (bp_ft_f < f_min_c) | (bp_ft_f > f_max_c) |
        (dPs > dPs_c) |
        (dPd > dPd_c) |
        (dT > dT_c) |
        (bp_ft_T > T)
    )
    
    corrupted_indx = valleys_indx[2:][mask]
            
    return corrupted_indx

In [13]:
#Loop preprocessing CST Processing

raw_data_dir = "F:\\Lab\\Research\\NIBP_PPG\\Dataset\\Ex_3\\RU_dataset\\CST_Raw"
down_folder_dir = "F:\\Lab\\Research\\NIBP_PPG\\Dataset\\Ex_3\\RU_dataset\\CST_Processed"
file_list = os.listdir(raw_data_dir)
for file_name in file_list:
    if file_name.endswith('.npz'):

        Data = np.load(raw_data_dir+"\\"+file_name)
        ppg1_raw_sig = Data['x1']
        ppg2_raw_sig = Data['x2']
        bp_raw_sig = Data['y']

        #Nan 제거
        ppg1_raw_sig = Nan_interpolation(ppg1_raw_sig)
        ppg2_raw_sig = Nan_interpolation(ppg2_raw_sig)
        bp_raw_sig = Nan_interpolation(bp_raw_sig)

        #Lag compensation using Cross Correlation function
        ppg1_fp = ppg1_raw_sig[0:250]
        ppg1_fp = (ppg1_fp-np.mean(ppg1_fp))/np.std(ppg1_fp)
        bp_fp = bp_raw_sig[0:250]
        bp_fp = (bp_fp-np.mean(bp_fp))/np.std(bp_fp)
        correlation = np.correlate(ppg1_fp, bp_fp, mode='same')
        max_indx = signal.find_peaks(correlation[125:])
        ppg1_sig_lc = ppg1_raw_sig[max_indx[0][0]:]
        ppg2_sig_lc = ppg2_raw_sig[max_indx[0][0]:]
        bp_raw_sig = bp_raw_sig[:-max_indx[0][0]]

        #Frequency filter
        # FIR Filter 사용
        numtaps = 1001
        ppg1_sig_lc = BPFilter(ppg1_sig_lc, numtaps = numtaps, window = 'hamming')
        ppg2_sig_lc = BPFilter(ppg2_sig_lc, numtaps = numtaps, window = 'hamming')
        """ppg1_sig_lc = ppg1_sig_lc[(numtaps-1)//2:]
        ppg2_sig_lc = ppg2_sig_lc[(numtaps-1)//2:]"""
        bp_MAF_sig = MAFilter(bp_raw_sig, 10)[(numtaps-1)//2:]
        
        #신호 길이 맞춰주기
        gap = ppg1_sig_lc.shape[0] - bp_MAF_sig.shape[0]
        if gap < 0:
            bp_MAF_sig = bp_MAF_sig[:ppg1_sig_lc.shape[0]]
        if gap > 0:
            ppg1_sig_lc = ppg1_sig_lc[:bp_MAF_sig.shape[0]]
            ppg2_sig_lc = ppg2_sig_lc[:bp_MAF_sig.shape[0]]

        #Downsample to 25Hz
        ori_rate = 125
        targ_rate = 25
        decimation_factor = int(ori_rate / targ_rate)
        ppg1_sig = decimate(ppg1_sig_lc, decimation_factor, zero_phase=True)
        ppg2_sig = decimate(ppg2_sig_lc, decimation_factor, zero_phase=True)

        #5초간 윈도우
        indx_num_ppg = 5 * 25
        num_chunks_ppg = ppg1_sig.shape[0] // indx_num_ppg

        indx_num_bp = 5 * 125
        num_chunks_bp = bp_MAF_sig.shape[0] // indx_num_bp

        ppg1_data = ppg1_sig[:num_chunks_ppg * indx_num_ppg].reshape(num_chunks_ppg, indx_num_ppg)
        ppg2_data = ppg2_sig[:num_chunks_ppg * indx_num_ppg].reshape(num_chunks_ppg, indx_num_ppg)
        bp_data = bp_MAF_sig[:num_chunks_bp * indx_num_bp].reshape(num_chunks_bp, indx_num_bp)

        #Normalization
        ppg1_data_mean = np.mean(ppg1_data, axis=1, keepdims=True)
        ppg1_data_std = np.std(ppg1_data, axis=1, keepdims=True)
        ppg1_data = (ppg1_data - ppg1_data_mean) / ppg1_data_std
        
        ppg2_data_mean = np.mean(ppg2_data, axis=1, keepdims=True)
        ppg2_data_std = np.std(ppg2_data, axis=1, keepdims=True)
        ppg2_data = (ppg2_data - ppg2_data_mean) / ppg2_data_std

        #Auto-correlation - ppg_corrupted_sig - Auto-correlation 값으로 줄세워서 가장 주기성 높은 애들로만 7200개 모으기
        #auto_correlation 배열에 두번째 correlation peak 값 저장
        ppg1_data_corrupted_indx = np.empty((0,), dtype=int)
        auto_correlation = np.empty((0,))

        for i in range(ppg1_data.shape[0]):
            auto_correlation_array = np.correlate(ppg1_data[i],ppg1_data[i],mode='same')
            auto_correlation_array = auto_correlation_array/np.max(auto_correlation_array)
            temp_peaks = signal.find_peaks(auto_correlation_array, height=0.7)

            if temp_peaks[0].shape[0] < 3 or temp_peaks[0].shape[0] > 5:
                ppg1_data_corrupted_indx = np.append(ppg1_data_corrupted_indx, i)
                auto_correlation = np.append(auto_correlation,0)
            else:
                sec_peak_indx = temp_peaks[0][temp_peaks[0].shape[0]//2+1]
                auto_correlation = np.append(auto_correlation, auto_correlation_array[sec_peak_indx])

        #BP SAI corrupted signal filter
        bp_corrupted_indx = SAI_Algorithm(bp_MAF_sig)

        bp_data_corrupted_indx = np.empty((0,), dtype = int)
        for i in range(bp_data.shape[0]):
            ispeak, _ = signal.find_peaks(bp_data[i], distance = 60)
            if len(ispeak) <= 2 or len(ispeak) >=10:
                bp_data_corrupted_indx = np.append(bp_data_corrupted_indx, i)
                continue

            else:
                for k in range(bp_corrupted_indx.shape[0]):
                    if bp_corrupted_indx[k]>625*i and bp_corrupted_indx[k]<625*(i+1):
                        bp_data_corrupted_indx = np.append(bp_data_corrupted_indx, i)
                        break

        corrupted_data_indx = np.concatenate((ppg1_data_corrupted_indx, bp_data_corrupted_indx))
        corrupted_data_indx = np.unique(corrupted_data_indx)

        ppg1_data = np.delete(ppg1_data, corrupted_data_indx, axis=0)
        ppg2_data = np.delete(ppg2_data, corrupted_data_indx, axis=0)
        bp_data = np.delete(bp_data, corrupted_data_indx, axis=0)
        auto_correlation = np.delete(auto_correlation, corrupted_data_indx, axis=0)
        auto_correlation_indx = np.argsort(auto_correlation)[::-1]

        #Sort according to correlation value index
        ppg1_data = ppg1_data[auto_correlation_indx]
        ppg2_data = ppg2_data[auto_correlation_indx]
        bp_data = bp_data[auto_correlation_indx]

        #Calculate mean of blood pressures
        bp_data_label = np.empty((0,2))
        for i in range(bp_data.shape[0]):
            peaks_temp,_ = signal.find_peaks(bp_data[i], distance = 60)
            valleys_temp = np.empty((0,),dtype=int)
            valleys_temp = np.append(valleys_temp, [np.argmin(bp_data[i, peaks_temp[k]: peaks_temp[k + 1]]) + peaks_temp[k] for k in range(len(peaks_temp) - 1)])
            peaks_mean = np.mean(bp_data[i,peaks_temp])
            valleys_mean = np.mean(bp_data[i, valleys_temp])
            bp_data_label = np.insert(bp_data_label, bp_data_label.shape[0], [peaks_mean, valleys_mean], axis=0)

        #처리한 데이터 저장
        print(file_name+" successfully processed, ppg data size: ", ppg1_data.shape, ppg2_data.shape, 'blood pressure size: ', bp_data_label.shape)
        npz_filename = "processed_"+file_name[4:]
        np.savez(down_folder_dir+"\\"+npz_filename, x1=ppg1_data, x2=ppg2_data, y=bp_data_label)

        """gap = ppg1_data.shape[0]-bp_data_label.shape[0]

        if gap < 0:
            np.savez(down_folder_dir+"\\"+npz_filename, x1=ppg1_data[:ppg1_data.shape[0]], x2=ppg2_data[:ppg1_data.shape[0]], y=bp_data_label[:ppg1_data.shape[0]])
        if gap > 0:
            np.savez(down_folder_dir+"\\"+npz_filename, x1=ppg1_data[:bp_data_label.shape[0]], x2=ppg2_data[:bp_data_label.shape[0]], y=bp_data_label[:bp_data_label.shape[0]])"""

raw_RBDB_11.npz successfully processed, ppg data size:  (124, 125) (124, 125) blood pressure size:  (124, 2)
raw_RBDB_12.npz successfully processed, ppg data size:  (56, 125) (56, 125) blood pressure size:  (56, 2)
raw_RBDB_13.npz successfully processed, ppg data size:  (187, 125) (187, 125) blood pressure size:  (187, 2)
raw_RBDB_14.npz successfully processed, ppg data size:  (185, 125) (185, 125) blood pressure size:  (185, 2)
raw_RBDB_15.npz successfully processed, ppg data size:  (116, 125) (116, 125) blood pressure size:  (116, 2)
raw_RBDB_16.npz successfully processed, ppg data size:  (421, 125) (421, 125) blood pressure size:  (421, 2)
raw_RBDB_17.npz successfully processed, ppg data size:  (224, 125) (224, 125) blood pressure size:  (224, 2)
raw_RBDB_18.npz successfully processed, ppg data size:  (65, 125) (65, 125) blood pressure size:  (65, 2)
raw_RBDB_19.npz successfully processed, ppg data size:  (82, 125) (82, 125) blood pressure size:  (82, 2)
raw_RBDB_1_.npz successfully

In [8]:
Data.close()